<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/99-05-rnn-lot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# RNN-lot


In [ ]:
# !pip install tensorflow-gpu==2.0.0
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

# 마지막 회차번호

In [ ]:
# 마지막 회차 번호
noEnd ='992' 

url = f'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={noEnd}'

# user-agent
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [ ]:
result = requests.get(url, headers={"user-agent":user_agent})

In [ ]:
result

In [ ]:
f = BytesIO(result.content)
# df = pd.read_excel(f,sheet_name='excel',skiprows=3)
html_result = pd.read_html(f)

In [ ]:
df_raw = pd.DataFrame(html_result[1])
# df = df.drop([0])
df_raw.head()

In [ ]:
# columns = [col for i, col in enumerate(df.iloc[0])]
# df.columns = columns
# columns

# column name 설정
col_names = [fn+sn if fn != sn else fn for fn, sn in zip(df_raw.iloc[0], df_raw.iloc[1])]  
df_raw.columns = col_names

# 0, 1행 삭제
df_lot = df_raw.drop([0,1], axis=0)

# 컬럼 형변환
df_lot = df_lot.astype({'회차':'uint', '당첨번호1':'uint', '당첨번호2':'uint', '당첨번호3':'uint', '당첨번호4':'uint', '당첨번호5':'uint', '당첨번호6':'uint', '당첨번호보너스':'uint'}, copy=True)

# 당첨금액 형변환
df_lot['1등당첨금액'] = df_lot['1등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['2등당첨금액'] = df_lot['2등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['3등당첨금액'] = df_lot['3등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['4등당첨금액'] = df_lot['4등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['5등당첨금액'] = df_lot['5등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')

# 회차기준 오름차준 정렬
df_lot = df_lot.sort_values('회차', ascending=True)

# 회차필드 index로 설정
df_lot.set_index('회차', inplace=True)

# 필요없는 컬럼 삭제
df_lot.drop(['년도', '추첨일', '1등당첨자수', '2등당첨자수', '3등당첨자수', '4등당첨자수', '5등당첨자수'], axis=1, inplace=True)

df_lot.head(5)

In [ ]:
df_lot.info()

In [ ]:
import numpy as np

# 당첨번호를 원핫인코딩벡터로 변환
def lotto2onehot(numbers):
    onehot = np.zeros(45)
    for number in numbers:
        onehot[int(number)-1] = 1 #로또번호에 해당하는 인덱스에 1로 설정 
    return onehot.tolist()

# 원핫인코딩벡터를 당첨번호로 변환
def onehot2lotto(onehots):
    numbers = [i+1 for i, val in enumerate(onehots) if val == 1]
    return numbers

In [ ]:
# 함수 동작 확인

lotto = [1,3,5,7,9,11]
onehot_lotto = lotto2onehot(lotto)
print(onehot_lotto)
onehot2lotto(onehot_lotto)

In [ ]:
# 당첨번호 및 보너스번호 추출
lotto_numbers = df_lot[['당첨번호1','당첨번호2','당첨번호3', '당첨번호4', '당첨번호5', '당첨번호6']].to_numpy()
bonus_number = df_lot['당첨번호보너스'].to_numpy()
lotto_numbers

In [ ]:
# 당첨번호 onehot 인코딩
onehot_lotto_numbers = list(map(lotto2onehot, lotto_numbers))

In [ ]:
print(onehot2lotto(onehot_lotto_numbers[-1]))

In [ ]:
# make X and Y data
WINDOW_SIZE = 10
x_data = []
y_data = []
for i in range(len(onehot_lotto_numbers)-WINDOW_SIZE):
    x_data.append([onehot_lotto_numbers[i:i+WINDOW_SIZE-1]])
    y_data.append(onehot_lotto_numbers[i+WINDOW_SIZE])

print(onehot2lotto(y_data[-1]))

In [ ]:
print(f'x_data.shape: {np.shape(x_data)}, y_data.shape: {np.shape(y_data)}')

In [ ]:
# numpy and reshape
x_data = np.reshape(x_data, (-1,WINDOW_SIZE-1, 45))
y_data = np.reshape(y_data, (-1, 45))
print(f'x_data.shape: {x_data.shape}, y_data.shape: {y_data.shape}')

In [ ]:
# x_data = tf.constant(x_data)
# y_data = tf.constant(y_data)

In [ ]:
# 데이터셋 분리
train_idx = (0, 800)
valid_idx = (801, 900)
test_idx = (900, len(y_data))

x_train = x_data[train_idx[0]:train_idx[1]]
y_train = y_data[train_idx[0]:train_idx[1]]
x_val = x_data[valid_idx[0]:valid_idx[1]]
y_val = y_data[valid_idx[0]:valid_idx[1]]
x_test = x_data[test_idx[0]:test_idx[1]]
y_test = y_data[test_idx[0]:test_idx[1]]


bonus_train = bonus_number[train_idx[0]:train_idx[1]]
bonus_val = bonus_number[valid_idx[0]:valid_idx[1]]
bonus_test = bonus_number[test_idx[0]:test_idx[1]]

In [ ]:
x_train[1,1,:]

In [ ]:
y_train[0,:]

In [ ]:
# LSTM 모델 생성

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

rnn_units = 7

# 모델을 정의합니다.
model = keras.Sequential([              
    keras.layers.LSTM(rnn_units, batch_input_shape=(None, WINDOW_SIZE-1, 45), return_sequences=False),
    # keras.layers.LSTM(rnn_units, batch_input_shape=(None, WINDOW_SIZE-1, 45), return_sequences=True),
    # keras.layers.LSTM(rnn_units, return_sequences=True),
    # keras.layers.LSTM(rnn_units, return_sequences=False),
    keras.layers.Dense(45, activation='relu')
])

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
import os
import time

# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

keras_callback = [                  
  tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True),
  tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30, mode='min', min_delta=0.0001, restore_best_weights=True)
]
# checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_prefix,
#     save_weights_only=True)

# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30, mode='min', min_delta=0.0001, restore_best_weights=True)

In [ ]:
EPOCHS = 200
history = model.fit(x_data, 
                    y_data, 
                    epochs=EPOCHS, 
                    batch_size = 64, 
                    validation_data=(x_val, y_val),
                    verbose=0, 
                    callbacks=[keras_callback])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
plt.figure(figsize=(20,20))

acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')

acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
acc_ax.plot(history.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
# 88 회부터 현재까지 1등부터 5등까지 상금 평균
mean_prize = [df_lot['1등당첨금액'].iloc[87:].mean(), 
              df_lot['2등당첨금액'].iloc[87:].mean(), 
              df_lot['3등당첨금액'].iloc[87:].mean(), 
              df_lot['4등당첨금액'].iloc[87:].mean(), 
              df_lot['5등당첨금액'].iloc[87:].mean()]
print(mean_prize)

In [ ]:
# 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

In [ ]:
import random
def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    random.shuffle(ball_box)

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    selected_balls.sort()
    
    return selected_balls

In [ ]:
def calc_reward_from_dataset(one_hot_true_numbers, true_bonuses, one_hot_pred_numbers):
  rewards = []
  grades = np.zeros(6, dtype=int)

  for i, pn in enumerate(one_hot_pred_numbers):
    true_numbers = onehot2lotto(one_hot_true_numbers[i])
    true_bonus = true_bonuses[i]

    pred_numbers = get_lotto_from_predicted_number(one_hot_pred_numbers[i])
    grade, sum_reward = calc_reward(true_numbers, true_bonus, pred_numbers)
    grades[grade] += 1
  
    sum_reward = 0
    for j in range(10): # 10판 수행
      pred_numbers = gen_numbers_from_probability(one_hot_pred_numbers[i])
      grade, reward = calc_reward(true_numbers, true_bonus, pred_numbers)
      sum_reward += reward
      grades[grade] += 1
  
    rewards.append(sum_reward)

  return grades, rewards

In [ ]:
def get_lotto_from_predicted_number(pred_lotto):
  lotto = []
  for i in range(6):
    lotto.append(np.argmax(pred_lotto)+1)
    pred_lotto[np.argmax(pred_lotto)] = 0 # reset

  return np.array(sorted(lotto))

In [ ]:
# train, valid, test 데이터로  predict 하여 reward 성능치 확인해보자
y_total_pred =  model.predict(x_data)
total_grades, total_rewards = calc_reward_from_dataset(y_data, bonus_number, y_total_pred)

print('1st  2nd  3rd  4th  5th  6th  Rewards')
print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d}'.format(total_grades[0], total_grades[1], total_grades[2], total_grades[3], total_grades[4], total_grades[5], int(np.sum(total_rewards))))


In [ ]:
y_train_pred = model.predict(x_train)
train_grades, train_rewards = calc_reward_from_dataset(y_train, bonus_train, y_train_pred)
print('1st  2nd  3rd  4th  5th  6th  Rewards')
print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d}'.format(train_grades[0], train_grades[1], train_grades[2], train_grades[3], train_grades[4], train_grades[5], int(np.sum(train_rewards))))


In [ ]:
y_val_pred = model.predict(x_val)
val_grades, val_rewards = calc_reward_from_dataset(y_val, bonus_val, y_val_pred)

print('1st  2nd  3rd  4th  5th  6th  Rewards')
print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d}'.format(val_grades[0], val_grades[1], val_grades[2], val_grades[3], val_grades[4], val_grades[5], int(np.sum(val_rewards))))


In [ ]:
y_test_pred = model.predict(x_test)
test_grades, test_rewards = calc_reward_from_dataset(y_test, bonus_test, y_test_pred)

print('1st  2nd  3rd  4th  5th  6th  Rewards')
print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d}'.format(test_grades[0], test_grades[1], test_grades[2], test_grades[3], test_grades[4], test_grades[5], int(np.sum(test_rewards))))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

total_reward = train_rewards + val_rewards + test_rewards

plt.plot(total_reward)
plt.ylabel('rewards')
plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

ax = plt.figure().gca()
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

rewards = [sum(train_rewards), sum(val_rewards), sum(test_rewards)]

class_color=['green', 'blue', 'red']

plt.bar(['train', 'val', 'test'], rewards, color=class_color)
plt.ylabel('rewards')
plt.show()

In [ ]:
previous_lotto = np.reshape(x_data[-1,:,:], (1, WINDOW_SIZE-1, 45))

In [ ]:
pred_lotto = model.predict(previous_lotto)

In [ ]:
pred_lotto[0]

In [ ]:
# prediction 확률이 높은 6개의 번호
get_lotto_from_predicted_number(pred_lotto[0])

In [ ]:
# 확률기준으로 100개의 ball을 생성한 후 6개의 번호 렌덤하게 선택
gen_numbers_from_probability(pred_lotto[0])